In [1]:
!pip install unsloth datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 27.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 14.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of

In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from tqdm import tqdm
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothBCOTrainer: No module named 'UnslothBCOTrainer'. Using tempfile instead!


In [3]:
# Load your LoRA fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./checkpoint-65",  # Replace with your local fine-tuned checkpoint path
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.4.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.138 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

Unsloth: Will load ./checkpoint-65 as a legacy tokenizer.
Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

In [7]:
# 1. Load TruthfulQA
truthfulqa = load_dataset("truthfulqa/truthful_qa", "generation")["validation"]

README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [8]:
# 2. Convert to Open-Instruct format (only using best answer)
formatted_eval_data = []
 
for sample in tqdm(truthfulqa, desc="Formatting TruthfulQA"):
    question = sample["question"]
    best_answer = sample["best_answer"]
    formatted_eval_data.append({
        "instruction": question,
        "input": "",
        "output": best_answer
    })

Formatting TruthfulQA: 100%|██████████| 817/817 [00:00<00:00, 11605.78it/s]


In [9]:
# 3. Convert to a DataFrame
df_eval = pd.DataFrame(formatted_eval_data)
df_eval.to_json("truthfulqa_openinstruct_eval.json", orient="records", lines=True)
 
print("✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'")

✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'


In [10]:
# Load the evaluation dataset
with open("truthfulqa_openinstruct_eval.json", "r") as f:
    eval_data = [json.loads(line) for line in f]
 

In [11]:
# Run inference and store predictions
model_outputs = []
for sample in tqdm(eval_data, desc="Running inference"):
    prompt = f"### Instruction:\n{sample['instruction']}\n\n### Response:\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=150)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the generated part
    if "### Response:" in decoded_output:
        generated = decoded_output.split("### Response:")[-1].strip()
    else:
        generated = decoded_output.strip()
    model_outputs.append({
        "instruction": sample["instruction"],
        "best_answer": sample["output"],
        "model_output": generated
    })
 
# Save model outputs
with open("MODEL_RESPONSES.json", "w") as f:
    json.dump(model_outputs, f, indent=2)
 
print("✅ Inference completed and saved to 'model_truthfulqa_outputs.json'")

Running inference: 100%|██████████| 817/817 [1:08:38<00:00,  5.04s/it]

✅ Inference completed and saved to 'model_truthfulqa_outputs.json'


In [12]:
!pip install matplotlib networkx langchain langchain-community 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 148.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 163.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 200.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 190.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 145.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import re
import json
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
 
# === Load Ollama LLM ===
llm = Ollama(model="llama3.1:8b-instruct-q4_K_M")